In [1]:
import os
import pandas
import Bio
import Bio.Seq
import Bio.SeqIO
import Bio.SeqRecord

In [9]:
input_filepath = "2_references/library2_sequences.csv"
output_filepath = "2_references/references/references.fasta"
os.makedirs(os.path.dirname(output_filepath), exist_ok=True)

# the adapter on the 5' end of the sequence needs to be removed
adaptor_5end = 'ACGACGCTCTTCCGATCT'
# the adapter on the 5' end of the sequence needs to be added
adaptor_3end = 'CATAGACAAACAGGCATGCAAG'

In [5]:
# Open reference file
ref_df = pandas.read_csv(input_filepath)
# check that all entries are unique
assert(len(ref_df['id'].unique())==len(ref_df['id']))
# check that all sequences are unique
assert(len(ref_df['seq'].unique())==len(ref_df['seq']))

In [6]:
# Make column with an adjusted reference sequence
ref_df['seq_adjusted'] = ''
for index, row in ref_df.iterrows():
    seq = row['seq']
    start_trim_pos = len(adaptor_5end)
    assert(seq[:start_trim_pos]==adaptor_5end)
    ref_df.loc[index, 'seq_adjusted'] = seq[start_trim_pos:] + adaptor_3end

In [10]:
# Write fasta file
seq_records = []
for index, row in ref_df.iterrows():
    seq_records.append(
        Bio.SeqRecord.SeqRecord(
            Bio.Seq.Seq(row['seq_adjusted']),
            id=str(row['id']),
            description="",
        )
    )

with open(output_filepath, "w") as output_handle:
    Bio.SeqIO.write(seq_records, output_handle, "fasta")

In [11]:
# are all values in seq_adjusted_3UTR unique?
assert(len(ref_df['seq_adjusted'].unique())==len(ref_df['seq_adjusted']))